# 笔记

## 概括(AI 生成+自己完善)

本教程深入介绍了 PyTorch 的自动微分引擎 `torch.autograd` 的核心机制和用法。教程首先阐明了反向传播算法中梯度计算的重要性，并展示了如何通过设置张量的 `requires_grad=True` 属性来让 Autograd 跟踪其上的操作，从而构建计算图。关键概念如 `grad_fn` (记录梯度函数) 和 `backward()` (启动梯度计算) 得到了详细的解释。教程强调了梯度会累积在叶子张量的 `.grad` 属性中，并介绍了如何通过 `torch.no_grad()` 或 `tensor.detach()` 方法来局部禁用梯度跟踪，这对于模型评估或冻结参数非常有用。此外，教程还简要提及了计算图的动态性以及针对非标量输出计算雅可比向量积的概念。

## 关键函数、语法(自己总结+AI 优化)

- `tensor_instance = torch.[creation_op](..., requires_grad=True)` 或 `tensor_instance.requires_grad_(True)`:
    -   Tensor属性或方法，用于指示 Autograd 跟踪对此张量的操作以计算梯度。
    -   对于定义模型参数或任何需要计算其相对于某个标量（例如损失）的梯度的张量至关重要。
    -   例如: `w = torch.randn(5, 3, requires_grad=True)`。
- `tensor.grad_fn`:
    -   计算图中张量的一个属性，引用创建此张量作为操作结果的 `Function` 对象（梯度函数）。
    -   叶子张量（例如直接创建的模型参数）和 `requires_grad=False` 的张量的 `grad_fn` 为 `None`，前者是因为张量不是由函数得到的，后者是因为张量不需要计算梯度。
- `loss_tensor.backward(gradient=None, retain_graph=False, create_graph=False)`:
    -   从 `loss_tensor` 开始，通过计算图启动反向传播。
    -   `gradient`: 对于标量 `loss_tensor`（最常见情况），可以省略此参数（默认为 `torch.tensor(1.0)`）。对于非标量 `loss_tensor`，`gradient` 必须是形状相同的张量，用来计算雅可比-向量积。
    -   `retain_graph=True`: 如果需要在同一图或其部分上多次调用 `backward()`（例如，对于多个输出或图的部分被重用），则必须设置此参数。默认 (`False`) 情况下，为了效率，图在 `backward()` 后会被释放掉。
    -   `create_graph=True`: 如果设置，则允许通过构建反向传播的图来计算高阶导数。
- `tensor.grad`:
    -   是张量（通常是 `requires_grad=True` 的叶子张量）的一个属性，在调用 `backward()` 后累积 `loss_tensor` 相对于此张量的梯度总和。
    -   梯度是累加的；在每个新的梯度计算周期之前，必须显式地将它们清零（例如，通过 `optimizer.zero_grad()` 或 `tensor.grad.zero_()`）。
- `tensor.detach()`:
    -   创建一个新张量，该张量与原始张量共享相同的基础数据（存储），但从当前计算图中**分离**出来。
    -   新张量的 `requires_grad` 将为 `False`。对一个张量中数据的修改**会影响另一个张量**。
- `tensor.grad.zero_()`:
    -   一个原地操作方法，用于将张量的 `.grad` 属性设置为零。
    -   在不使用优化器或针对特定张量手动管理梯度时很有用。
- `torch.nn.functional.binary_cross_entropy_with_logits(input, target, weight=None, size_average=None, reduce=None, reduction='mean', pos_weight=None)`:
    -   一种损失函数，它将 Sigmoid 层和 BCELoss（二元交叉熵损失）结合在一个类中。
    -   此版本比使用普通的 Sigmoid 后跟 BCELoss 在数值上更稳定。
    -   `input` 是原始的、未经 Sigmoid 激活的 logits。
    -   `target` 是二元标签（通常为0或1）。

## 提问与解答(自己提问+AI 回答)

- **问题1 (关于 "Computing Gradients" 和梯度累积的说明):**
  教程提到 "PyTorch **accumulates the gradients**"。
    1.  为什么梯度累积是 PyTorch Autograd 的**默认行为**？这种设计有什么实际的好处或应用场景（除了在标准训练循环中必须手动清零之外）？
    2.  教程中还提到使用 `inp.grad.zero_()` 来清零梯度。在之前的教程中，我们使用 `optimizer.zero_grad()`。这两种方法在功能和使用上有什么主要区别？

  - **解答:**
    1.  **梯度累积的默认行为及好处:**
        -   **原因与好处**: PyTorch 默认累积梯度是为了提供更大的灵活性。一个主要的应用场景是**模拟更大的批量大小 (batch size)**。如果因为 GPU 内存限制无法使用大的批量，可以将一个小批量的梯度计算多次（每次都是新的数据，但不执行 `optimizer.step()`），并将它们的梯度累积起来，然后用累积的梯度执行一次参数更新。这等效于使用这些小批量总和大小的一个大批量进行训练。
        -   **其他场景**:
            -   **自定义梯度操作**: 在某些高级应用中，可能需要对来自不同损失分量或不同部分的梯度进行加权或修改，累积机制使得这成为可能。
            -   **多任务学习**: 如果一个模型有多个损失函数，可以分别对每个损失调用 `backward()`，它们的梯度会累积到共享参数上，然后进行一次优化步骤。

    2.  **`tensor.grad.zero_()` 与 `optimizer.zero_grad()` 的区别:**
        -   **`tensor.grad.zero_()`**: 这是一个直接作用于单个张量的 `.grad` 属性的原地操作。它将特定张量的梯度清零。如果模型有很多参数，你需要手动遍历所有参数并对每个参数的 `.grad` 调用 `zero_()`。
        -   **`optimizer.zero_grad()`**: 这个方法属于优化器对象 (例如 `torch.optim.SGD` 的实例)。当你创建优化器时，你将模型的参数 (例如 `model.parameters()`) 传递给了它。调用 `optimizer.zero_grad()` 会**遍历在创建优化器时注册给它的所有参数，并将它们各自的 `.grad` 属性清零**。
        -   **主要区别**:
            -   **范围**: `optimizer.zero_grad()` 作用于优化器管理的所有参数，而 `tensor.grad.zero_()` 只作用于单个张量。
            -   **便利性**: 对于整个模型的参数梯度清零，`optimizer.zero_grad()` 显然更方便和常用。
            -   **使用场景**: 在标准的模型训练循环中，总是使用 `optimizer.zero_grad()`。如果出于某种特殊原因需要单独控制某个特定张量的梯度清零（可能在不使用优化器或进行非常规梯度操作时），才会直接使用 `tensor.grad.zero_()`。

    参考资料:
    -   PyTorch 官方文档关于 Autograd 的说明: [https://pytorch.org/docs/stable/notes/autograd.html](https://pytorch.org/docs/stable/notes/autograd.html) (特别是关于梯度累积的部分)

- **问题2 (关于 "Disabling Gradient Tracking" 当中的 `torch.no_grad()` 与 `detach()`):**
  教程介绍了 `with torch.no_grad():` 和 `tensor.detach()` 两种禁用梯度跟踪的方式。
    1.  这两种方法在作用范围、目的和典型使用场景上有何主要区别？
    2.  假设有一个张量 `A` 且 `A.requires_grad=True`。我们创建 `B = A.detach()`，然后 `C = B * 2`。此时 `C.requires_grad` 是什么？如果后续操作使得 `C` 对某个最终损失有贡献，并且我们尝试反向传播，梯度是否会流回 `A`？为什么？

  - **解答:**
    1.  **`torch.no_grad()` 与 `tensor.detach()` 的区别:**
        -   **`torch.no_grad()`**:
            -   **作用范围**: 这是一个上下文管理器。在其 `with` 块内部执行的所有 PyTorch 操作都不会被 Autograd 跟踪，即使输入的张量原本设置了 `requires_grad=True`。在这个块内新创建的张量，其 `requires_grad` 属性默认为 `False`。
            -   **目的**: 主要用于在不需要计算梯度的阶段（如模型评估/推理、或者只是想执行一些不会影响梯度计算的辅助计算）来节省内存和加速计算。它完全关闭了该代码块的梯度计算功能。
            -   **典型使用场景**: 模型验证循环 (`model.eval()` 模式下)，或者在训练循环中进行一些不希望影响梯度的监控或数据处理。
        -   **`tensor.detach()`**:
            -   **作用范围**: 这是一个张量方法。它创建一个与原张量共享数据存储的新张量，但这个新张量从当前的计算图中被“分离”出来。
            -   **目的**: 创建一个不参与梯度计算但仍与原数据相关的张量。新张量的 `requires_grad` 属性将是 `False`。如果原张量 `requires_grad=True`，`detach()` 会切断反向传播到原张量的路径（通过这个新张量）。
            -   **典型使用场景**:
                -   当你想将一个需要梯度的张量用于某个不需要梯度的计算，但又不希望这个计算影响到原张量的梯度流。
                -   在某些强化学习场景或 GANs 的训练中，可能需要将一个网络的输出作为另一个网络的输入，但不想让梯度流过第一个网络。
                -   从计算图中取出一个张量的值用于绘图或记录，而不保留其梯度历史。

    2.  **`detach()` 后梯度流的行为:**
        -   `A.requires_grad = True`
        -   `B = A.detach()`: 此时 `B` 与 `A` 共享数据，但 `B.requires_grad` 为 `False`。`B` 已经从计算图中分离，不再有 `grad_fn`。
        -   `C = B * 2`: 由于 `B.requires_grad` 是 `False`，运算 `* 2` 也不会被跟踪，所以 `C.requires_grad` 也是 `False`。
        -   **梯度流**: 如果后续操作使得 `C` 对某个最终损失有贡献，并且通过某种方式（例如 `C_prime = C.clone().requires_grad_(True)`）让 `C` 的一个副本参与到后续的梯度计算中，当反向传播时，梯度**不会**流回 `A`。因为 `B` 是通过 `detach()` 从 `A` 分离出来的，`A` 和 `B` 之间的计算图连接已经被切断。Autograd 无法通过 `B` 或 `C` 找到回到 `A` 的路径来计算梯度。

    参考资料:
    -   PyTorch 官方文档 `torch.no_grad`: [https://pytorch.org/docs/stable/generated/torch.no_grad.html](https://pytorch.org/docs/stable/generated/torch.no_grad.html)
    -   PyTorch 官方文档 `torch.Tensor.detach`: [https://pytorch.org/docs/stable/generated/torch.Tensor.detach.html](https://pytorch.org/docs/stable/generated/torch.Tensor.detach.html)

- **问题3 (关于教程中 "Optional Reading: Tensor Gradients and Jacobian Products" 这一部分):**
  教程解释了对于向量函数 $\vec{y}=f(\vec{x})$，PyTorch 可以通过 `y.backward(v)` 计算雅可比向量积 $v^T \cdot J$。
    1.  在哪些实际的深度学习高级应用或研究场景中，直接计算或利用雅可比向量积（而不是标量损失的梯度）会更加合适？
    2.  教程示例中 `out.backward(torch.ones_like(out), retain_graph=True)` 使用 `torch.ones_like(out)` 作为向量 `v`。这种选择（全1向量）在计算雅可比向量积 $v^T \cdot J$ 时，实际上得到了什么？它与直接对 `out` 的每个元素求和然后计算梯度有什么关系？

  - **解答:**
    1.  **雅可比向量积 (JVP) 的高级应用场景:**
        -   **Hessian-vector products (HVP)**: 二阶导数（Hessian 矩阵）在某些优化算法（如牛顿法、共轭梯度法）和模型分析中非常重要。直接计算和存储完整的 Hessian 矩阵对于大模型来说非常昂贵。然而，通常只需要 Hessian 矩阵与某个向量的乘积 (HVP)，即 $H \cdot v$。JVP 可以用来高效计算 HVP，因为 $H \cdot v = \frac{\partial (\nabla_{\vec{x}} L)^T \cdot v}{\partial \vec{x}}$，其中 $(\nabla_{\vec{x}} L)^T \cdot v$ 是一个标量，可以对其再次求导。
        -   **敏感性分析/影响力函数**: 分析模型输出对输入的敏感性，或者某个训练样本对模型参数的影响，可能涉及到雅可比矩阵或其乘积的计算。
        -   **某些正则化技术**: 例如，雅可比正则化，直接对输入的雅可比矩阵的范数进行惩罚，以鼓励模型学习更平滑的函数。
        -   **生成对抗网络 (GANs)**: 在一些 GAN 的变体或分析中，可能需要计算鉴别器输出相对于输入的梯度，这本身就是一个向量，后续可能需要JVP。
        -   **物理启发式神经网络 (PINNs)**: 在求解偏微分方程时，可能需要计算网络输出相对于其输入的导数，并将其作为损失函数的一部分。

    2.  **`v = torch.ones_like(out)` 的含义:**
        -   如果 `out` 是一个向量 (或更一般地，一个张量) $\vec{y} = (y_1, y_2, ..., y_m)$，那么 $v = (1, 1, ..., 1)$。
        -   此时，雅可比向量积 $v^T \cdot J$ 为:
            $$ (1, 1, ..., 1) \cdot \begin{pmatrix}
            \frac{\partial y_1}{\partial x_1} & \cdots & \frac{\partial y_1}{\partial x_n} \\
            \vdots & \ddots & \vdots \\
            \frac{\partial y_m}{\partial x_1} & \cdots & \frac{\partial y_m}{\partial x_n}
            \end{pmatrix} = \left( \sum_{i=1}^m \frac{\partial y_i}{\partial x_1}, \sum_{i=1}^m \frac{\partial y_i}{\partial x_2}, ..., \sum_{i=1}^m \frac{\partial y_i}{\partial x_n} \right) $$
        -   这实际上计算的是 **`out` 中所有元素之和** $S = \sum y_i$ 相对于输入 $\vec{x}$ 中每个元素 $x_j$ 的梯度 $(\frac{\partial S}{\partial x_1}, ..., \frac{\partial S}{\partial x_n})$。
        -   这是因为 $\frac{\partial S}{\partial x_j} = \frac{\partial (\sum y_i)}{\partial x_j} = \sum_i \frac{\partial y_i}{\partial x_j}$。
        -   因此，`out.backward(torch.ones_like(out))` 等效于先计算 `scalar_sum = out.sum()`，然后调用 `scalar_sum.backward()`。这是当你想得到 `out` 中所有元素对输入的总贡献的梯度时的一种常见做法。

    参考资料:
    -   PyTorch Autograd Mechanics (官方文档深入探讨): [https://pytorch.org/docs/stable/notes/autograd.html](https://pytorch.org/docs/stable/notes/autograd.html)


[Learn the Basics](intro.html) \|\|
[Quickstart](quickstart_tutorial.html) \|\|
[Tensors](tensorqs_tutorial.html) \|\| [Datasets &
DataLoaders](data_tutorial.html) \|\|
[Transforms](transforms_tutorial.html) \|\| [Build
Model](buildmodel_tutorial.html) \|\| **Autograd** \|\|
[Optimization](optimization_tutorial.html) \|\| [Save & Load
Model](saveloadrun_tutorial.html)

Automatic Differentiation with `torch.autograd`
===============================================

When training neural networks, the most frequently used algorithm is
**back propagation**. In this algorithm, parameters (model weights) are
adjusted according to the **gradient** of the loss function with respect
to the given parameter.

To compute those gradients, PyTorch has a built-in differentiation
engine called `torch.autograd`. It supports automatic computation of
gradient for any computational graph.

Consider the simplest one-layer neural network, with input `x`,
parameters `w` and `b`, and some loss function. It can be defined in
PyTorch in the following manner:


In [1]:
import torch

x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

Tensors, Functions and Computational graph
==========================================

This code defines the following **computational graph**:

![](https://pytorch.org/tutorials/_static/img/basics/comp-graph.png)

In this network, `w` and `b` are **parameters**, which we need to
optimize. Thus, we need to be able to compute the gradients of loss
function with respect to those variables. In order to do that, we set
the `requires_grad` property of those tensors.


<div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>NOTE:</strong></div>

<div style="background-color: #f3f4f7; padding-left: 10px; padding-top: 10px; padding-bottom: 10px; padding-right: 10px">

<p>You can set the value of <code>requires_grad</code> when creating atensor, or later by using <code>x.requires_grad_(True)</code> method.</p>

</div>



A function that we apply to tensors to construct computational graph is
in fact an object of class `Function`. This object knows how to compute
the function in the *forward* direction, and also how to compute its
derivative during the *backward propagation* step. A reference to the
backward propagation function is stored in `grad_fn` property of a
tensor. You can find more information of `Function` [in the
documentation](https://pytorch.org/docs/stable/autograd.html#function).


In [2]:
print(f"Gradient function for z = {z.grad_fn}")
print(f"Gradient function for loss = {loss.grad_fn}")

Gradient function for z = <AddBackward0 object at 0x0000027A23C25960>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x0000027A23C25BA0>


Computing Gradients
===================

To optimize weights of parameters in the neural network, we need to
compute the derivatives of our loss function with respect to parameters,
namely, we need $\frac{\partial loss}{\partial w}$ and
$\frac{\partial loss}{\partial b}$ under some fixed values of `x` and
`y`. To compute those derivatives, we call `loss.backward()`, and then
retrieve the values from `w.grad` and `b.grad`:


In [ ]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.2735, 0.0137, 0.0545],
        [0.2735, 0.0137, 0.0545],
        [0.2735, 0.0137, 0.0545],
        [0.2735, 0.0137, 0.0545],
        [0.2735, 0.0137, 0.0545]])
tensor([0.2735, 0.0137, 0.0545])


<div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>NOTE:</strong></div>

<div style="background-color: #f3f4f7; padding-left: 10px; padding-top: 10px; padding-bottom: 10px; padding-right: 10px">

<ul>
<li>We can only obtain the <code>grad</code> properties for the leafnodes of the computational graph, which have <code>requires_grad</code> propertyset to <code>True</code>. For all other nodes in our graph, gradients will not beavailable.- We can only perform gradient calculations using<code>backward</code> once on a given graph, for performance reasons. If we needto do several <code>backward</code> calls on the same graph, we need to pass<code>retain_graph=True</code> to the <code>backward</code> call.</li>
</ul>
```

</div>



Disabling Gradient Tracking
===========================

By default, all tensors with `requires_grad=True` are tracking their
computational history and support gradient computation. However, there
are some cases when we do not need to do that, for example, when we have
trained the model and just want to apply it to some input data, i.e. we
only want to do *forward* computations through the network. We can stop
tracking computations by surrounding our computation code with
`torch.no_grad()` block:


In [4]:
z = torch.matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)

True
False


Another way to achieve the same result is to use the `detach()` method
on the tensor:


In [5]:
z = torch.matmul(x, w)+b
z_det = z.detach()
print(z_det.requires_grad)

False


There are reasons you might want to disable gradient tracking:

:   -   To mark some parameters in your neural network as **frozen
        parameters**.
    -   To **speed up computations** when you are only doing forward
        pass, because computations on tensors that do not track
        gradients would be more efficient.


More on Computational Graphs
============================

Conceptually, autograd keeps a record of data (tensors) and all executed
operations (along with the resulting new tensors) in a directed acyclic
graph (DAG) consisting of
[Function](https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function)
objects. In this DAG, leaves are the input tensors, roots are the output
tensors. By tracing this graph from roots to leaves, you can
automatically compute the gradients using the chain rule.

In a forward pass, autograd does two things simultaneously:

-   run the requested operation to compute a resulting tensor
-   maintain the operation's *gradient function* in the DAG.

The backward pass kicks off when `.backward()` is called on the DAG
root. `autograd` then:

-   computes the gradients from each `.grad_fn`,
-   accumulates them in the respective tensor's `.grad` attribute
-   using the chain rule, propagates all the way to the leaf tensors.

<div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>NOTE:</strong></div>

<div style="background-color: #f3f4f7; padding-left: 10px; padding-top: 10px; padding-bottom: 10px; padding-right: 10px">

<p>An important thing to note is that the graph is recreated from scratch; after each<code>.backward()</code> call, autograd starts populating a new graph. This isexactly what allows you to use control flow statements in your model;you can change the shape, size and operations at every iteration ifneeded.</p>

</div>



Optional Reading: Tensor Gradients and Jacobian Products
========================================================

In many cases, we have a scalar loss function, and we need to compute
the gradient with respect to some parameters. However, there are cases
when the output function is an arbitrary tensor. In this case, PyTorch
allows you to compute so-called **Jacobian product**, and not the actual
gradient.

For a vector function $\vec{y}=f(\vec{x})$, where
$\vec{x}=\langle x_1,\dots,x_n\rangle$ and
$\vec{y}=\langle y_1,\dots,y_m\rangle$, a gradient of $\vec{y}$ with
respect to $\vec{x}$ is given by **Jacobian matrix**:

$$\begin{aligned}
J=\left(\begin{array}{ccc}
   \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{1}}{\partial x_{n}}\\
   \vdots & \ddots & \vdots\\
   \frac{\partial y_{m}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
   \end{array}\right)
\end{aligned}$$

Instead of computing the Jacobian matrix itself, PyTorch allows you to
compute **Jacobian Product** $v^T\cdot J$ for a given input vector
$v=(v_1 \dots v_m)$. This is achieved by calling `backward` with $v$ as
an argument. The size of $v$ should be the same as the size of the
original tensor, with respect to which we want to compute the product:


In [6]:
inp = torch.eye(4, 5, requires_grad=True)
out = (inp+1).pow(2).t()
out.backward(torch.ones_like(out), retain_graph=True)
print(f"First call\n{inp.grad}")
out.backward(torch.ones_like(out), retain_graph=True)
print(f"\nSecond call\n{inp.grad}")
inp.grad.zero_()
out.backward(torch.ones_like(out), retain_graph=True)
print(f"\nCall after zeroing gradients\n{inp.grad}")

First call
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.]])

Second call
tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.]])

Call after zeroing gradients
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.]])


Notice that when we call `backward` for the second time with the same
argument, the value of the gradient is different. This happens because
when doing `backward` propagation, PyTorch **accumulates the
gradients**, i.e. the value of computed gradients is added to the `grad`
property of all leaf nodes of computational graph. If you want to
compute the proper gradients, you need to zero out the `grad` property
before. In real-life training an *optimizer* helps us to do this.


<div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>NOTE:</strong></div>

<div style="background-color: #f3f4f7; padding-left: 10px; padding-top: 10px; padding-bottom: 10px; padding-right: 10px">

<p>Previously we were calling <code>backward()</code> function without parameters. This is essentially equivalent to calling<code>backward(torch.tensor(1.0))</code>, which is a useful way to compute the gradients in case of a scalar-valued function, such as loss during neural network training.</p>

</div>



------------------------------------------------------------------------


Further Reading
===============

-   [Autograd
    Mechanics](https://pytorch.org/docs/stable/notes/autograd.html)
